In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-midterm-p4/train.csv
/kaggle/input/2022-ai-midterm-p4/test.csv
/kaggle/input/2022-ai-midterm-p4/submit_sample.csv


In [2]:
#랜덤시드 고정
import random
import torch

seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
#gpu 사용가능하면 사용하는 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
#데이터 로드
sample = pd.read_csv('/kaggle/input/2022-ai-midterm-p4/submit_sample.csv')
train = pd.read_csv('/kaggle/input/2022-ai-midterm-p4/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-midterm-p4/test.csv')

In [5]:
#데이터 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1000 non-null   int64  
 1   hour                    1000 non-null   int64  
 2   hour_bef_temperature    1000 non-null   float64
 3   hour_bef_precipitation  1000 non-null   float64
 4   hour_bef_windspeed      1000 non-null   float64
 5   hour_bef_humidity       1000 non-null   float64
 6   hour_bef_visibility     1000 non-null   float64
 7   hour_bef_ozone          1000 non-null   float64
 8   hour_bef_pm10           1000 non-null   float64
 9   hour_bef_pm2.5          1000 non-null   float64
 10  count                   1000 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 86.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtyp

In [6]:
train

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
995,1647,14,20.4,0.0,3.6,12.0,2000.0,0.047,23.0,13.0,108.0
996,1649,15,18.5,0.0,4.1,24.0,849.0,0.064,92.0,83.0,78.0
997,1650,4,16.1,0.0,0.6,60.0,2000.0,0.028,39.0,28.0,27.0
998,1652,16,17.7,0.0,2.3,32.0,1949.0,0.037,64.0,22.0,118.0


In [7]:
#데이터 전처리
xtrain = train.drop(['id','count'],axis=1)
ytrain = train['count']
xtest = test.drop(['id'],axis=1)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

xtrain_std = sc.fit_transform(xtrain)
xtest_std = sc.transform(xtest)

In [8]:
# X_train = torch.FloatTensor(np.array(xtrain)).to(device)
# y_train = torch.FloatTensor(np.array(ytrain)).to(device)
# X_test = torch.FloatTensor(np.array(xtest)).to(device)
X_train = torch.FloatTensor(np.array(xtrain_std)).to(device)
y_train = torch.FloatTensor(np.array(ytrain)).to(device)
X_test = torch.FloatTensor(np.array(xtest_std)).to(device)

In [9]:
#모델 빌드에 필요한 설정값 확인
print(X_train.shape)
print(y_train.shape)

torch.Size([1000, 9])
torch.Size([1000])


In [10]:
#모델 빌드
import torch.nn as nn

class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(9, 512, bias=True)
        self.linear2 = nn.Linear(512, 512, bias=True)
        self.linear3 = nn.Linear(512, 512, bias=True)
        self.linear4 = nn.Linear(512, 1, bias=True)

        self.active = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)
        nn.init.xavier_normal_(self.linear3.weight)
        nn.init.xavier_normal_(self.linear4.weight)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.active(x)
        x = self.dropout(x)
        x = self.linear4(x)
        
        return x

In [11]:
#모델 정의
model = DNN().to(device)

In [12]:
import torch.optim as optim

#Optimizer 정의
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#Loss 정의 (회귀문제이므로 MSELoss)
loss = nn.MSELoss()

In [13]:
#모델 학습
epochs = 10000
for epoch in range(epochs + 1):
    
    optimizer.zero_grad()
    
    hypothesis = model(X_train)
    
    cost = loss(hypothesis, y_train.unsqueeze(1))
    
    cost.backward()
    
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(epoch, cost.item())

0 19233.349609375
1000 435.4714660644531
2000 177.73379516601562
3000 108.16580200195312
4000 71.62747192382812
5000 70.63478088378906
6000 77.67394256591797
7000 56.495262145996094
8000 43.20412063598633
9000 43.591331481933594
10000 36.53710174560547


In [14]:
#학습한 모델을 이용해서 Test 데이터 예측
with torch.no_grad():
    model.eval()
    predict = model(X_test)

In [15]:
#예측한 값을 csv로 변환
sample['count'] = predict.cpu().detach().numpy()
sample.to_csv('submit.csv',index=False)